In [1]:
import numpy as np
root = '../../'
caffe_root = root + 'caffe/'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe
import os
import random

In [2]:
DATA_DIR='../../dataset/i-LIDS/data/'
file_list_a=os.listdir(DATA_DIR)

In [3]:
name_dict={}
for name in file_list_a:
    if name[-3:]=='jpg':
        id = name[:4]
        view = int(name[4:7])
        if not name_dict.has_key(id):
            name_dict[id]=[]
            name_dict[id].append([])
            name_dict[id].append([])
            name_dict[id].append([])
            name_dict[id].append([])
            name_dict[id].append([])
            name_dict[id].append([])
            name_dict[id].append([])
            name_dict[id].append([])
        name_dict[id][view-1].append(DATA_DIR+name)  

print len(name_dict)
print name_dict['0010']

119
[['../../dataset/i-LIDS/data/0010001.jpg'], ['../../dataset/i-LIDS/data/0010002.jpg'], ['../../dataset/i-LIDS/data/0010003.jpg'], ['../../dataset/i-LIDS/data/0010004.jpg'], [], [], [], []]


In [4]:
# choose test ids:
test_id_number = 59
set_no = 1
phase = 'test'
filename_test = '../../dataset/i-LIDS/exp_set/testid%03d_set%02d_%s.txt'%(test_id_number,(set_no),phase)
print filename_test

file_object = open(filename_test)
try:
    all_the_text = file_object.read( )
finally:
    file_object.close( )

test_dict = {}

lines = all_the_text.split('\n')
for filename in lines:
    if filename!='':
        if name_dict.has_key(filename):
            test_dict[filename] = name_dict[filename]
print len(test_dict)

../../dataset/i-LIDS/exp_set/testid059_set01_test.txt
59


In [5]:
def readList(name_list,name_dict): 
    import random
    import os
    probes=[]
    gallerys=[]
    for key in name_list:
        rand_view = random.randint(0,len(name_dict[key])-1)
        #print rand_view,name_dict[key][rand_view]
        while len(name_dict[key][rand_view])==0:
            rand_view = random.randint(0,len(name_dict[key])-1)
        
        rand_view_pair = random.randint(0,len(name_dict[key])-1)
        #print rand_view_pair,name_dict[key][rand_view_pair]
        while len(name_dict[key][rand_view_pair])==0 or rand_view_pair == rand_view:
            rand_view_pair = random.randint(0,len(name_dict[key])-1)
        probes.append(name_dict[key][rand_view][0])
        gallerys.append(name_dict[key][rand_view_pair][0])
        
    if len(probes)!=len(gallerys):
        print('something wrong! list length does not match!/n')
        return 0
    else:
        return probes,gallerys
def generateScoreList(net,probes,gallerys):
    transformer = caffe.io.Transformer({'data': (net.blobs['data'].data.shape)})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', np.array([ 104.00698793,  116.66876762,  122.67891434])) # mean pixel
    transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
    transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB
    
    scoreList=[]
    N,C,H,W=net.blobs['data'].data.shape
    from time import clock
    start=clock()
    #galleryData is same for each probe
    galleryLen=len(gallerys)
    galleryDataList=[]
    for galleryIdx in range(galleryLen):
        galleryName=gallerys[galleryIdx]
        galleryImage=transformer.preprocess('data', caffe.io.load_image(galleryName))
        galleryDataList.append(galleryImage)
        galleryIdx+=1
    #galleryData and probeData
    galleryData=np.asarray(galleryDataList)
    probeData=np.zeros((galleryLen,C,H,W))
    
    net.blobs['data'].reshape(galleryLen,C,H,W)
    net.blobs['data_p'].reshape(galleryLen,C,H,W)
    #process each probe
    for probeIdx in range(len(probes)):
        probeName=probes[probeIdx]
        probeImage=transformer.preprocess('data', caffe.io.load_image(probeName))
        #batch data assignment
        probeData[:,:,:,:]=probeImage
        net.blobs['data'].data[:] = probeData
        net.blobs['data_p'].data[:] = galleryData
        #net forwad
        net.forward()
        #get output score
        outScore=net.blobs['softmax_score'].data[:,(0,1)]    #softmax_score[0] and softmax_score[1]
        score_sum=np.exp(outScore[:,0]*1.0)+np.exp(outScore[:,1]*1.0)
        similarScore=outScore[:,1]#np.exp(outScore[:,1]*1.0)/score_sum
        #scoreList.append each probe score
        scoreList.append(similarScore.tolist())
        if (probeIdx+1)%10==0:
            sys.stdout.write('\r%3d/%d, '%(probeIdx+1,len(probes))+probeName)
            sys.stdout.flush()
    #we get scoreList, then cal predictLists
    predictLists=[]
    for score in scoreList:
        probeRankList=np.argsort(score)[::-1]
        predictLists.append(probeRankList)
    finish=clock()
    print('\r  Processing %dx%d pairs cost %f second time'%(len(probes),len(gallerys),(finish-start)))
    return scoreList,predictLists

def calCMC(net,set_no,rand_times=10):
    from cmc import evaluateCMC


    #rand 10 times for stable result
    cmc_list=[]
    for i in range(rand_times):
        print 'Round %d with rand list:'%i
        probes,gallerys=readList(list_name)
        scoreList,predictLists=generateScoreList(net,probes,gallerys)
        gtLabels=range(len(probes))
        cmc=evaluateCMC(gtLabels,predictLists)
        cmc_list.append(cmc)
    return np.average(cmc_list,axis=0)

def getCVPRcmc():
    #return the cmc values, 100 dim vetor
    import numpy as np
    cmcIndex=[0,4,8,12,16,21,25,29,33,37,41,45,49,53]
    cmcOfCVPRImproved=[0.5474,0.8753,0.9293,0.9712,0.9764,0.9811,0.9899,0.9901,0.9912,0.9922,0.9937,0.9945,0.9951,1]
    pOfCVPRImproved = np.poly1d(np.polyfit(cmcIndex,cmcOfCVPRImproved,10))
    x_line=range(50)
    cmc=pOfCVPRImproved(x_line)
    return cmc


In [6]:
MODEL_FILE = '../../experiments/reid_earlyfusion_google_bigmap/set01/deploy.prototxt'
PRETRAINED = '../../experiments/reid_earlyfusion/set01/Snapshots/set01_all_google_bigmap_iter_300000.caffemodel'
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Classifier(MODEL_FILE, PRETRAINED,caffe.TEST)

In [7]:
key_list = []
for key in test_dict.keys():
    key_list.append(key)

key_list_shuffle = key_list[:]
random.shuffle(key_list_shuffle)
probes,gallerys=readList(key_list_shuffle[:],test_dict)
print len(key_list)

59


In [8]:
from cmc import evaluateCMC
cmcDict = {}

In [9]:
cmc_list=[]
from cmc import evaluateCMC
for i in range(3):
    key_list_shuffle = key_list[:]
    random.shuffle(key_list_shuffle)
    probes,gallerys=readList(key_list_shuffle[:59],name_dict)
    scoreList,predictLists=generateScoreList(net,probes,gallerys)
    gtLabels=range(len(probes))
    cmc=evaluateCMC(gtLabels,predictLists)
    cmc_list.append(cmc)

  Processing 59x59 pairs cost 13.708461 second time
  Processing 59x59 pairs cost 13.336753 second time
  Processing 59x59 pairs cost 13.283191 second time


In [10]:
np.average(cmc_list,axis=0)

array([ 0.57062147,  0.71751412,  0.7740113 ,  0.82485876,  0.8700565 ,
        0.89265537,  0.91525424,  0.91525424,  0.92090395,  0.93220339,
        0.94350282,  0.94915254,  0.95480226,  0.96045198,  0.96045198,
        0.96610169,  0.97175141,  0.97740113,  0.97740113,  0.97740113,
        0.97740113,  0.97740113,  0.97740113,  0.98305085,  0.98305085,
        0.98870056,  0.98870056,  0.98870056,  0.98870056,  0.98870056,
        0.98870056,  0.99435028,  0.99435028,  0.99435028,  0.99435028,
        0.99435028,  0.99435028,  0.99435028,  0.99435028,  0.99435028,
        0.99435028,  0.99435028,  0.99435028,  0.99435028,  0.99435028,
        0.99435028,  0.99435028,  0.99435028,  0.99435028,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ])

In [119]:
np.average(cmc_list,axis=0)

array([ 0.51694915,  0.63050847,  0.6779661 ,  0.7220339 ,  0.75084746,
        0.77118644,  0.79491525,  0.81864407,  0.83898305,  0.84915254,
        0.86610169,  0.87457627,  0.88305085,  0.89661017,  0.90847458,
        0.91186441,  0.92372881,  0.92542373,  0.93728814,  0.94576271,
        0.94915254,  0.95423729,  0.95762712,  0.95932203,  0.96101695,
        0.96271186,  0.96271186,  0.96610169,  0.96949153,  0.97118644,
        0.97627119,  0.9779661 ,  0.97966102,  0.98135593,  0.98135593,
        0.98305085,  0.98305085,  0.98474576,  0.98474576,  0.98474576,
        0.98983051,  0.99322034,  0.99661017,  0.99661017,  0.99661017,
        0.99830508,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ])

In [47]:
cmcDict={}
cmcDict['CVPR15Imp']=getCVPRcmc()
cmcDict['Our_HNM']=cmc_all
#plot the dictionary, sorted by rank1 rate
plotCMC(cmcDict)

NameError: name 'cmc_all' is not defined

In [18]:
def getSIFT():
    #return the cmc values, 100 dim vetor
    import numpy as np
    cmcIndex=[0,4,8,12,16,21,25,29,33,37,41,45,49,53]
    cmcOfCVPRImproved=[0.3427,0.8753,0.9293,0.9712,0.9764,0.9811,0.9899,0.9901,0.9912,0.9922,0.9937,0.9945,0.9951,1]
    pOfCVPRImproved = np.poly1d(np.polyfit(cmcIndex,cmcOfCVPRImproved,10))
    x_line=range(50)
    cmc=pOfCVPRImproved(x_line)
    return cmc

def plotCMC_step5(cmcDict,pathname):
    import matplotlib.pyplot as plt
    get_ipython().magic(u'matplotlib inline')   
    from matplotlib.legend_handler import HandlerLine2D
    import numpy as np

    #plot the cmc curve, record CVPR from the pdf paper.cmc[0,4,8,12,16,21,25,29,33,37,41,45,49]
    rank2show=25
    rankStep=5
    cmcIndex=np.arange(0,rank2show,rankStep)   #0,5,10,15,20,25

    colorList=['rv-','g^-','bs-','yp-','c*-','m+-','kd-','gs-','b^-']
    #start to plot
    plt.ioff()
    fig = plt.figure(figsize=(6,5),dpi=180)
    sortedCmcDict = sorted(cmcDict.items(), key=lambda (k, v): v[1])[::-1]
    for idx in range(len(sortedCmcDict)):
        cmc_dictList=sortedCmcDict[idx]
        cmc_name=cmc_dictList[0]
        cmc_list=cmc_dictList[1]
        #print cmc_name,": ",cmc_list
        #x for plot
        x_point=[item+1 for item in cmcIndex]
        x_line=range(rank2show)
        x_plot=[temp+1 for temp in x_line]
        #start plot
        plt.plot(x_plot, cmc_list[x_line],colorList[idx],label="%02.02f%% %s"%(100*cmc_list[0],cmc_name))
        plt.plot(x_point,cmc_list[cmcIndex],colorList[idx]+'.')
        #plt.legend(loc=4,handler_map={line: HandlerLine2D(numpoints=1)})
        #idx of color +1
        idx+=1
    #something to render

    plt.xlabel('Rank')
    plt.ylabel('Identification Rate')
    plt.xticks(np.arange(0,rank2show+1,5))
    plt.yticks(np.arange(0,1.01,0.1))
    plt.grid()
    plt.legend(loc=4)
    plt.savefig(pathname)
    plt.show()

RuntimeError: Could not open file ../../experiments/reid_simplemodel/set03/deploy.prototxt